# PVAnalytics QA Process: Power

In [1]:
import pvanalytics
import numpy as np
import rdtools
from statistics import mode
import json
# pvanalytics.__version__
from pvanalytics.features.clearsky import reno       #update to just do a pvanalytics import?
import pvlib
import matplotlib.pyplot as plt
import pandas as pd
from pvanalytics.quality import data_shifts as ds
from pvanalytics.quality import gaps
from pvanalytics.quality.outliers import zscore
from pvanalytics.features.daytime import power_or_irradiance
from pvanalytics.quality.time import shifts_ruptures
from pvanalytics.features import daytime
from pvanalytics.system import (is_tracking_envelope,
                                infer_orientation_fit_pvwatts)
from pvanalytics.features.clipping import geometric
import ruptures as rpt
import os
import boto3

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 12,
                           'figure.figsize': [4.5, 3],
                           'lines.markeredgewidth': 0,
                           'lines.markersize': 2
                           })

In the following example, a process for assessing the data quality of the AC power data streams for system 2107 is shown, using PVAnalytics functions. This example pipeline illustrates how several PVAnalytics functions can be used in sequence to assess the quality of a power data stream.

First, we download and import the power data from a PV installation under the [2023 solar data prize data set](https://data.openei.org/s3_viewer?bucket=oedi-data-lake&limit=100&prefix=pvdaq%2F2023-solar-data-prize%2F). This data set is publicly available via the PVDAQ database in the DOE Open Energy Data Initiative (OEDI) (https://data.openei.org/submissions/4568), under system ID 2107. This data is timezone-localized.

In [2]:
with open('./data/2107_system_metadata.json', 'r') as f:
    metadata = json.load(f)

tz = metadata['System']['timezone_code']

def load_csv(file_path):
    df = pd.read_csv(
        file_path,
        index_col=0,
        parse_dates=True,
    )
    return df

df_elect = load_csv("./data/2107_electrical_data.csv")

df_elect = df_elect.tz_localize(tz, ambiguous=True)

power_columns = [x for x in df_elect.columns if 'power' in x]

latitude = metadata['Site']['latitude']
longitude = metadata['Site']['longitude']


In [ ]:
for col in power_columns:
    power_time_series = df_elect[col].copy()

    # Get the time frequency of the time series
    freq_minutes = mode(power_time_series.index.to_series().diff().dt.seconds / 60)
    data_freq = str(freq_minutes) + "min"
    power_time_series = power_time_series.asfreq(data_freq)    

    # REMOVE STALE DATA (that isn't during nighttime periods)
    # Day/night mask
    daytime_mask = power_or_irradiance(power_time_series)
    # Stale data mask
    stale_data_mask = gaps.stale_values_round(power_time_series,
                                              window=3,
                                              decimals=2)
    stale_data_mask = stale_data_mask & daytime_mask

    # REMOVE NEGATIVE DATA
    negative_mask = (power_time_series < 0)

    # FIND ABNORMAL PERIODS
    daily_min = power_time_series.resample('D').min()
    series_min = 0.1 * power_time_series.mean()
    erroneous_mask = (daily_min >= series_min)
    erroneous_mask = erroneous_mask.reindex(index=power_time_series.index,
                                            method='ffill',
                                            fill_value=False)

    # FIND OUTLIERS (Z-SCORE FILTER)
    zscore_outlier_mask = zscore(power_time_series, zmax=4,
                                 nan_policy='omit')

    # Get the percentage of data flagged for each issue, so it can later be logged
    pct_stale = round((len(power_time_series[
        stale_data_mask].dropna())/len(power_time_series.dropna())*100), 1)
    pct_negative = round((len(power_time_series[
        negative_mask].dropna())/len(power_time_series.dropna())*100), 1)
    pct_erroneous = round((len(power_time_series[
        erroneous_mask].dropna())/len(power_time_series.dropna())*100), 1)
    pct_outlier = round((len(power_time_series[
        zscore_outlier_mask].dropna())/len(power_time_series.dropna())*100), 1)



    # Filter the time series, taking out all of the issues
    issue_mask = ((~stale_data_mask) & (~negative_mask) &
              (~erroneous_mask) & (~zscore_outlier_mask))

    power_time_series = power_time_series[issue_mask].copy()
    power_time_series = power_time_series.asfreq(data_freq)


    # daily data completeness
    x = power_time_series.copy()
    x.loc[~daytime_mask] = 0
    data_completeness_score = gaps.completeness_score(x)


    # Trim the series based on daily completeness score
    trim_series = pvanalytics.quality.gaps.trim_incomplete(
        x, minimum_completeness=.25, freq=data_freq)

    power_time_series = power_time_series[trim_series].copy()
    power_time_series = power_time_series.asfreq(data_freq)

    # Get time of day from the associated datetime column
    time_of_day = pd.Series(power_time_series.index.hour +
                            power_time_series.index.minute/60,
                            index=power_time_series.index)
    # Pivot the dataframe
    dataframe = pd.DataFrame(pd.concat([power_time_series, time_of_day], axis=1))
    dataframe.columns = ["values", 'time_of_day']
    dataframe = dataframe.dropna()
    dataframe_pivoted = dataframe.pivot_table(index='time_of_day',
                                              columns=dataframe.index.date,
                                              values="values")

    # Get the modeled sunrise and sunset time series based on the system's
    # latitude-longitude coordinates
    modeled_sunrise_sunset_df = pvlib.solarposition.sun_rise_set_transit_spa(
         power_time_series.index, latitude, longitude)

    # Calculate the midday point between sunrise and sunset for each day
    # in the modeled irradiance series
    modeled_midday_series = modeled_sunrise_sunset_df['sunrise'] + \
        (modeled_sunrise_sunset_df['sunset'] -
         modeled_sunrise_sunset_df['sunrise']) / 2

    # Run day-night mask on the power time series
    daytime_mask = power_or_irradiance(power_time_series,
                                       freq=data_freq,
                                       low_value_threshold=.005)

    # Generate the sunrise, sunset, and halfway points for the data stream
    sunrise_series = daytime.get_sunrise(daytime_mask)
    sunset_series = daytime.get_sunset(daytime_mask)
    midday_series = sunrise_series + ((sunset_series - sunrise_series)/2)

    # Convert the midday and modeled midday series to daily values
    midday_series_daily, modeled_midday_series_daily = (
        midday_series.resample('D').mean(),
        modeled_midday_series.resample('D').mean())

    # Set midday value series as minutes since midnight, from midday datetime
    # values
    midday_series_daily = (midday_series_daily.dt.hour * 60 +
                           midday_series_daily.dt.minute +
                           midday_series_daily.dt.second / 60)
    modeled_midday_series_daily = \
        (modeled_midday_series_daily.dt.hour * 60 +
         modeled_midday_series_daily.dt.minute +
         modeled_midday_series_daily.dt.second / 60)

    # Estimate the time shifts by comparing the modelled midday point to the
    # measured midday point.
    is_shifted, time_shift_series = shifts_ruptures(midday_series_daily,
                                                    modeled_midday_series_daily,
                                                    period_min=15,
                                                    shift_min=15,
                                                    zscore_cutoff=1.5)

    # Create a midday difference series between modeled and measured midday, to
    # visualize time shifts. First, resample each time series to daily frequency,
    # and compare the data stream's daily halfway point to the modeled halfway
    # point
    midday_diff_series = (midday_series.resample('D').mean() -
                          modeled_midday_series.resample('D').mean()
                          ).dt.total_seconds() / 60

    # Generate boolean for detected time shifts
    if any(time_shift_series != 0):
        time_shifts_detected = True
    else:
        time_shifts_detected = False

    # Build a list of time shifts for re-indexing. We choose to use dicts.
    time_shift_series.index = pd.to_datetime(
        time_shift_series.index)
    changepoints = (time_shift_series != time_shift_series.shift(1))
    changepoints = changepoints[changepoints].index
    changepoint_amts = pd.Series(time_shift_series.loc[changepoints])
    time_shift_list = list()
    for idx in range(len(changepoint_amts)):
        if changepoint_amts[idx] == 0:
            change_amt = 0
        else:
            change_amt = -1 * changepoint_amts[idx]
        if idx < (len(changepoint_amts) - 1):
            time_shift_list.append({"datetime_start":
                                    str(changepoint_amts.index[idx]),
                                    "datetime_end":
                                        str(changepoint_amts.index[idx + 1]),
                                    "time_shift": change_amt})
        else:
            time_shift_list.append({"datetime_start":
                                    str(changepoint_amts.index[idx]),
                                    "datetime_end":
                                        str(time_shift_series.index.max()),
                                    "time_shift": change_amt})

    # Correct any time shifts in the time series
    new_index = pd.Series(power_time_series.index, index=power_time_series.index).dropna()
    for i in time_shift_list:
        if pd.notna(i['time_shift']):
            new_index[(power_time_series.index >= pd.to_datetime(i['datetime_start'])) &
                  (power_time_series.index < pd.to_datetime(i['datetime_end']))] = \
            power_time_series.index + pd.Timedelta(minutes=i['time_shift'])
    power_time_series.index = new_index

    # Remove duplicated indices and sort the time series (just in case)
    power_time_series = power_time_series[~power_time_series.index.duplicated(
        keep='first')].sort_index()

    # Set all values in the nighttime mask to 0
    power_time_series.loc[~daytime_mask] = 0
    # Resample the time series to daily mean
    power_time_series_daily = power_time_series.resample('D').mean()
    data_shift_start_date, data_shift_end_date = \
        ds.get_longest_shift_segment_dates(power_time_series_daily,
                                           use_default_models=False,
                                           method=rpt.Binseg, cost='rbf',
                                           penalty=15)
    data_shift_period_length = (data_shift_end_date -
                                data_shift_start_date).days

    # Get the number of shift dates
    data_shift_mask = ds.detect_data_shifts(power_time_series_daily,
                                            use_default_models=False,
                                            method=rpt.Binseg, cost='rbf',
                                            penalty=15)
    # Get the shift dates
    shift_dates = list(power_time_series_daily[data_shift_mask].index)
    if len(shift_dates) > 0:
        shift_found = True
    else:
        shift_found = False
    
    power_time_series = power_time_series[
        (power_time_series.index >=
         data_shift_start_date.tz_convert(power_time_series.index.tz)) &
        (power_time_series.index <=
         data_shift_end_date.tz_convert(power_time_series.index.tz))]

    power_time_series = power_time_series.asfreq(data_freq)
    power_time_series.to_csv(os.path.join("./data",  col + ".pkl"))
    
